# OLA 3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

finding the most relevant features

In [ ]:
df = pd.read_csv("../OLA-2/data/raw/heart_2020_cleaned_transformed.csv")

# Assuming df is your DataFrame, X is your feature matrix and y is your target variable
correlation = df.corr()

# Get correlation between each feature and target variable
corr_target = abs(correlation["HeartDisease"])

sorted_corr = corr_target.sort_values(ascending=False)
print(sorted_corr[:25])

Elbow curve

In [ ]:
# the most relevant features aer: GenHealth, AgeCategory, DiffWalking, Stroke, Diabetic, PhysicalHealth
features = df[
    ["GenHealth", "AgeCategory", "DiffWalking", "Stroke", "Diabetic", "PhysicalHealth"]
]

# Handle missing values if any. For instance, dropping rows with missing values
features.dropna(inplace=True)

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Elbow Method
distortions = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features_scaled)
    distortions.append(kmeans.inertia_)

plt.plot(range(2, 10), distortions, marker="o")
plt.xlabel("Number of clusters")
plt.ylabel("Distortion")
plt.show()

Silhouette score

In [ ]:
# Silhouette Score
# k = 6  # Choose the number of clusters based on the elbow diagram
# kmeans = KMeans(n_clusters=k, random_state=42)
# kmeans.fit(features_scaled)
# labels = kmeans.labels_

# score = silhouette_score(features_scaled, labels)
# print(f"Silhouette Score: {score}")

# # Silhouette Plot
# from sklearn.metrics import silhouette_samples

# silhouette_vals = silhouette_samples(features_scaled, labels)

# y_ax_lower, y_ax_upper = 0, 0
# yticks = []
# for i, c in enumerate(sorted(set(labels))):
#     c_silhouette_vals = silhouette_vals[labels == c]
#     c_silhouette_vals.sort()
#     y_ax_upper += len(c_silhouette_vals)
#     color = plt.cm.nipy_spectral(float(i) / len(set(labels)))
#     plt.barh(
#         range(y_ax_lower, y_ax_upper),
#         c_silhouette_vals,
#         height=1.0,
#         edgecolor="none",
#         color=color,
#     )
#     yticks.append((y_ax_lower + y_ax_upper) / 2.0)
#     y_ax_lower += len(c_silhouette_vals)

# silhouette_avg = np.mean(silhouette_vals)
# plt.axvline(silhouette_avg, color="red", linestyle="--")

# plt.yticks(yticks, [str(c) for c in sorted(set(labels))])
# plt.ylabel("Cluster")
# plt.xlabel("Silhouette score")
# plt.show()

# num_clusters = 6

# # Apply KMeans
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
# kmeans.fit(features_scaled)

# # Get the cluster labels for each data point
# labels = kmeans.labels_

# # Add the labels to the original DataFrame
# df['cluster'] = labels

# # Print the first few rows of the DataFrame to see the cluster labels
# print(df.head())

# # Visualizing the clusters is a bit tricky when you have more than two features.
# # One common approach is to use a pairplot, which shows scatter plots for each pair of features.
# # However, this requires all features to be numerical. If you have categorical features,
# # you'll need to handle them appropriately (e.g., by using one-hot encoding).
# # Here's an example of how you might create a pairplot using seaborn:

# import seaborn as sns
# sns.pairplot(df[['GenHealth', 'AgeCategory', 'DiffWalking', 'Stroke', 'Diabetic', 'cluster']], hue='cluster')
# plt.show()

kmeans

In [ ]:
from sklearn.cluster import KMeans

# Choose the number of clusters
num_clusters = 6

# Apply KMeans
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features_scaled)

# Get the cluster labels for each data point
labels = kmeans.labels_

# Add the labels to the original DataFrame
df["cluster"] = labels

# Print the first few rows of the DataFrame to see the cluster labels
df.head()

dbscan

In [ ]:
### too computational intensive
# from sklearn.cluster import DBSCAN

# # Apply DBSCAN
# # eps is the maximum distance between two samples for them to be considered as in the same neighborhood
# # min_samples is the minimum number of samples in a neighborhood for a point to be considered as a core point
# dbscan = DBSCAN(eps=0.5, min_samples=5)
# dbscan.fit(features_scaled)

# # Get the cluster labels for each data point
# labels = dbscan.labels_

# # Add the labels to the original DataFrame
# df["cluster"] = labels

# # Print the first few rows of the DataFrame to see the cluster labels
# print(df.head())

combining with logistic regression ???

In [ ]:
# Calculate average value of each feature for each cluster
for cluster in set(labels):
    cluster_data = features[labels == cluster]
    print(f"Cluster {cluster}:")
    print(cluster_data.mean())

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Add cluster labels as a new feature
df["cluster"] = labels

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("HeartDisease", axis=1), df["HeartDisease"], test_size=0.2, random_state=42
)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

visualize the clusters

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a pairplot of the features, colored by the cluster labels
sns.pairplot(df[['GenHealth', 'AgeCategory', 'DiffWalking', 'Stroke', 'Diabetic', 'cluster']], hue='cluster')

# Show the plot
plt.show()

hafla mistral:
    Discuss the limitations and assumptions of the clustering algorithms: Here's an example of how you can discuss the limitations and assumptions of KMeans:

KMeans is a widely used clustering algorithm that aims to partition the data into K distinct non-overlapping clusters such that each data point belongs to the cluster with the nearest mean. However, KMeans has several limitations and assumptions:

    KMeans assumes that the clusters are spherical and have similar densities, which might not be the case in real-world datasets.
    KMeans is sensitive to the initialization of the centroids, which can lead to different clustering results for different runs of the algorithm.
    KMeans is not suitable for categorical data, as it relies on the computation of the mean of the data points.
    KMeans requires the number of clusters to be specified in advance, which can be challenging to determine in some cases.

    Discuss the implications of the findings: Here's an example of how you can discuss the implications of the findings:

The clustering results suggest that there are distinct groups of patients with different risk profiles based on their characteristics. For example, Cluster 0 seems to be characterized by younger patients with good general health and no history of stroke or diabetes, while Cluster 1 seems to be characterized by older patients with poor general health and a history of stroke or diabetes. These findings could have important implications for the diagnosis and treatment of heart disease. For example, patients in Cluster 1 might benefit from more aggressive interventions to manage their risk factors, while patients in Cluster 0 might require less intensive interventions. However, further research is needed to validate these findings and to explore the potential clinical applications of the identified clusters.

    Proofread the notebook: Make sure to proofread the notebook to catch any spelling or grammar errors, and to ensure that the code runs correctly and the explanations are clear and concise. You might want to use a tool like Grammarly or Hemingway Editor to help you with proofreading.


Computational Cost: The Silhouette Score is computationally expensive, especially for large datasets. Your project's dataset is sizable, and calculating the Silhouette Score for various values of k can be time-consuming and resource-intensive.

Complexity for Multi-Dimensional Data: The Silhouette Score might not be as effective for datasets with many features, as it essentially measures how close each point in one cluster is to points in the neighboring clusters. This can be less interpretable when dealing with multi-dimensional data, where the notion of distance becomes less intuitive.

Potential Misleading Results in Certain Cases: In scenarios where clusters have varying densities or non-globular shapes, the Silhouette Score might not provide a clear indication of the best number of clusters. For heart disease data, which could have complex and overlapping patterns, the score might not reflect the true clustering tendency.

Sensitivity to Noise and Outliers: DBSCAN is a clustering algorithm designed to handle noise and outliers effectively by identifying them as separate from core clusters. The Silhouette Score, however, does not distinguish between noise/outliers and legitimate data points, potentially leading to misleading evaluations of cluster quality.

Elbow Method as a Simpler Alternative: The Elbow Method is a simpler and more computationally efficient approach. Although it might be less precise than the Silhouette Score, it can provide a good-enough estimate for the optimal number of clusters, which can be sufficient for many practical applications, including preliminary explorations in healthcare data.

Project Focus on DBSCAN: If your project's primary focus is on exploring DBSCAN as a clustering technique, which doesn't require pre-specifying the number of clusters, then the Silhouette Score loses its relevance, as it is generally more useful for algorithms like KMeans, where determining the number of clusters is crucial.